# Data Analysis and Schema Generation with TFDV

This Noteboooks helps performing Exploratory Data Analysis with BigQuery and TensorFlow Data Validation, and covers the following steps:

1. **Prepare the training dataset** with BigQuery.
2. **Generate statistics** from the training data.
3. **Visualise and analyse** the generated statistics.
4. **Infer** a **schema** from the generated statistics.
5. **Update** the schema with domain knowledge.
6. **Validate** the evaluation data against the schema.
7. **Save** the schema for later use.

The outputs from using this Data Analysis Notebook should be:
1. The **BigQuery SQL script** for Data Extraction and Preparation
2. The **Schema file** describing the final training dataset


### Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

WORKSPACE = 'workspace' # you can set to a GCS location
DATA_DIR = os.path.join(WORKSPACE, 'data')
RAW_SCHEMA_DIR = 'raw_schema'

if tf.io.gfile.exists(WORKSPACE):
    print("Removing previous workspace...")
    tf.io.gfile.rmtree(WORKSPACE)

print("Creating new workspace...")
tf.io.gfile.mkdir(WORKSPACE)
print("Creating data directory...")
tf.io.gfile.mkdir(DATA_DIR)

## Exploratory Data Analysis with BigQuery

### 1. Using BigQuery Magic

In [ ]:
%%bigquery

SELECT * 
FROM 
    sample_datasets.census
LIMIT 5;

In [ ]:
%%bigquery class_distribution

SELECT 
    income_bracket,
    count(*) count
FROM 
    sample_datasets.census
GROUP BY
    income_bracket
;

In [ ]:
class_distribution.plot(kind='bar', x='income_bracket', y='count');

In [ ]:
%%bigquery age_vs_education_num

SELECT DISTINCT
    age,
    capital_gain
FROM 
    sample_datasets.census
WHERE
    capital_gain < 20000
;

In [ ]:
age_vs_education_num.describe()

In [ ]:
plt.scatter(
    x=age_vs_education_num.age, 
    y=age_vs_education_num.capital_gain)

plt.show()

### 2. Write the final SQL Script

In [ ]:
SOURCE_QUERY = """
    SELECT 
        age,
        workclass,
        fnlwgt,
        education,
        education_num,
        marital_status,
        occupation,
        relationship,
        race,
        gender,
        capital_gain,
        capital_loss,
        hours_per_week,
        native_country,
        CASE WHEN income_bracket = ' <=50K' THEN 0 ELSE 1 END AS income_bracket
    FROM 
        @dataset_name.census
    WHERE
        age <= @age
"""

### 3. Using google.cloud.bigquery library

In [19]:
from google.cloud import bigquery

client = bigquery.Client()
sql = SOURCE_QUERY.replace('@age', '100').replace('@dataset_name', 'sample_dataset')
    
df = client.query(sql).to_dataframe()
df.head().T

,0,1,2,3,4
age,54,32,46,64,67
workclass,?,Private,Private,Private,Private
fnlwgt,148657,112137,225065,213391,142097
education,Preschool,Preschool,Preschool,9th,9th
education_num,1,1,1,5,5
marital_status,Married-civ-spouse,Married-civ-spouse,Married-civ-spouse,Married-civ-spouse,Married-civ-spouse
occupation,?,Machine-op-inspct,Machine-op-inspct,Other-service,Priv-house-serv
relationship,Wife,Wife,Wife,Wife,Wife
race,White,Asian-Pac-Islander,White,Black,White
gender,Female,Female,Female,Female,Female


In [20]:
from modules import sql_utils

In [21]:
sql = sql_utils.generate_source_query(age=100)
print(sql)


    SELECT 
        age,
        workclass,
        fnlwgt,
        education,
        education_num,
        marital_status,
        occupation,
        relationship,
        race,
        gender,
        capital_gain,
        capital_loss,
        hours_per_week,
        native_country,
        CASE WHEN income_bracket = ' <=50K' THEN 0 ELSE 1 END AS income_bracket
    FROM 
        sample_datasets.census
    WHERE
        age <= 100



### 4. Save data to CSV

In [ ]:
TRAIN_DATA_FILE = os.path.join(DATA_DIR,'train.csv')
df.to_csv(TRAIN_DATA_FILE, index=False)

In [ ]:
!wc -l $TRAIN_DATA_FILE 
!head $TRAIN_DATA_FILE 

## Tensorflow Data Validation for Schema Generation

In [ ]:
import tensorflow_data_validation as tfdv

TARGET_FEATURE_NAME = 'income_bracket'
WEIGHT_FEATURE_NAME = 'fnlwgt'

## 1. Compute Statistics

In [ ]:
train_stats = tfdv.generate_statistics_from_csv(
    data_location=TRAIN_DATA_FILE, 
    column_names=None, # CSV data file include header
    stats_options=tfdv.StatsOptions(
        weight_feature=WEIGHT_FEATURE_NAME,
        label_feature=TARGET_FEATURE_NAME,
        sample_rate=1.0
    )
)

In [ ]:
tfdv.visualize_statistics(train_stats)

## 2. Infer Schema

In [ ]:
schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema=schema)

## 3. Alter the Schema

In [ ]:
# Relax the minimum fraction of values that must come from the domain for feature occupation.
occupation = tfdv.get_feature(schema, 'occupation')
occupation.distribution_constraints.min_domain_mass = 0.9

# Add new value to the domain of feature native_country.
native_country_domain = tfdv.get_domain(schema, 'native_country')
native_country_domain.value.append('Egypt')

# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('EVALUATION')
schema.default_environment.append('SERVING')

# Specify that the class feature is not in SERVING environment.
tfdv.get_feature(schema, TARGET_FEATURE_NAME).not_in_environment.append('SERVING')

## 4. Save the Schema

In [ ]:
import shutil

if os.path.exists(RAW_SCHEMA_DIR):
    shutil.rmtree(RAW_SCHEMA_DIR)
    
os.mkdir(RAW_SCHEMA_DIR)

raw_schema_location = os.path.join(RAW_SCHEMA_DIR, 'schema.pbtxt')
tfdv.write_schema_text(schema, raw_schema_location)

### Test loading saved schema

In [ ]:
tfdv.load_schema_text(raw_schema_location)